In [ ]:
import pandas as pd
import numpy as np
import shutil
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from plotly import express as px
import cv2

In [ ]:
DATA_PATH = '../data/butteauxpeintres2/'

folders = next(os.walk(DATA_PATH))[1]  # Get a list of all subdirectories
NUM_CLASSES = len(folders)  # Count the number of subdirectories

IMG_SIZE = (256, 256)

## Define data pipeline

In [ ]:
dataset_train, dataset_test = image_dataset_from_directory(
                    DATA_PATH,
                    labels='inferred',
                    label_mode='categorical',
                    class_names=None,
                    color_mode='rgb',
                    batch_size=4,
                    image_size=IMG_SIZE,
                    shuffle=True,
                    seed=0,
                    subset='both',
                    validation_split=0.3,
                    interpolation='bilinear',
                    follow_links=False,
                    crop_to_aspect_ratio=True,
                )

class_names = dataset_train.class_names

In [ ]:
dataset_test.class_names

# Augmentations

In [ ]:
# Define the preprocessing layers for data augmentation
data_augmentation = tf.keras.Sequential([
  
  tf.keras.layers.RandomRotation(0.2), 
  tf.keras.layers.RandomZoom(0.2),
  tf.keras.layers.RandomContrast(0.2),
  tf.keras.layers.RandomBrightness(0.2),
])

dataset_train = dataset_train.map(lambda x, y: (data_augmentation(x), y))
dataset_train = dataset_train.prefetch(tf.data.AUTOTUNE)

# Define model

In [ ]:
# Define the input shape of the images
input_shape = IMG_SIZE + (3,)

# Define the convolutional neural network architecture
model = models.Sequential()
model.add(tf.keras.layers.Rescaling(1./255, input_shape=input_shape))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'],
              )

# Train

In [ ]:
# Train the model
history = model.fit(dataset_train, epochs=50, validation_data=dataset_test)

# Predict

In [ ]:
ds_test = [exs for exs in dataset_test.as_numpy_iterator()]
len(ds_test)

In [ ]:
for exs in dataset_test.as_numpy_iterator() : 
    
    X,Y = exs

    for x,y in zip(X,Y) : 

        print('='*20)

        label = class_names[np.argmax(y)]

        fig = px.imshow(x)
        fig.show()

        img = np.expand_dims(x, axis=0) # add batch dimension
        res = model.predict(img)

        print(f'truth : {label}')
        print(f'predict : {class_names[np.argmax(res)]}')

        fig2 = px.bar(x=class_names, y=res[0])
        fig2.show()

In [ ]:
X.shape